In [2]:
import pandas as pd
import numpy as np

In [3]:
from keras.models import load_model

Using TensorFlow backend.


In [6]:
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func

In [8]:
import requests
from requests_oauthlib import OAuth1

In [9]:
import cryptography
from cryptography.fernet import Fernet

In [29]:
import datetime as dt

In [22]:
from Candidates import candidates_list

In [4]:
model = load_model("candidate_classifier.h5")

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.


In [11]:
with open ('../config/config_key.key', 'rb') as ck:
    fernet_key = ck.read()

with open('../config/config_encrypt_1.key', 'rb') as c1:
    cke_e = c1.read()
with open('../config/config_encrypt_2.key', 'rb') as c2:
    cse_e = c2.read()
with open('../config/config_encrypt_3.key', 'rb') as c3:
    ate_e = c3.read()
with open('../config/config_encrypt_4.key', 'rb') as c4:
    atse_e = c4.read()

In [12]:
fernet = Fernet(fernet_key)

In [13]:
cke_d = fernet.decrypt(cke_e)
cse_d = fernet.decrypt(cse_e)
ate_d = fernet.decrypt(ate_e)
atse_d = fernet.decrypt(atse_e)

In [14]:
ck = cke_d.decode()
cs = cse_d.decode()
at = ate_d.decode()
ats = atse_d.decode()

In [15]:
# Create authorization object
auth = OAuth1(ck, cs, at, ats)

In [16]:
callback_url = "https://tweetocracy.herokuapp.com/"
payload = {
    'oauth_callback':callback_url
}

In [17]:
# Execute a POST/Auth request to twittier api to intitiate access
r = requests.post('https://api.twitter.com/oauth/request_token', auth = auth, data = payload)

In [18]:
print(f'Post Request Token URL:{r.url}')
print(f'Post Request Status:{r.status_code}')
print(f'Post Request Text: {r.text}')

Post Request Token URL:https://api.twitter.com/oauth/request_token
Post Request Status:200
Post Request Text: oauth_token=HkUHbQAAAAAA_QraAAABbJXe_SA&oauth_token_secret=EVwmMddyaTvoriDN6h6gHxGAlx0QR01x&oauth_callback_confirmed=true


In [19]:
# Collect response information
response_output = r.text
# Relevant paramters are received as a string, separated by an '&' character
response_parameters = response_output.split("&")

In [20]:
# Store relevant response paramters in variables
oauth_token = response_parameters[0][12:]
print(f'OAuth_token:{oauth_token}')
oauth_token_secret=response_parameters[1][19:]
print(f'Oauth Token Secret:{oauth_token_secret}')
oauth_callback_confirmed = bool(response_parameters[2][25:])
print(f'Callback Confirmed:{oauth_callback_confirmed}')

OAuth_token:HkUHbQAAAAAA_QraAAABbJXe_SA
Oauth Token Secret:EVwmMddyaTvoriDN6h6gHxGAlx0QR01x
Callback Confirmed:True


In [21]:
extended_payload = {
    'tweet_mode': 'extended'
}

In [23]:
candidate_name = candidates_list[0]['name']
candidate_id = candidates_list[0]["twitter_user_id"]

In [24]:
user_get = requests.get(f'https://api.twitter.com/1.1/statuses/user_timeline.json?id={candidate_id}&count=100', params = extended_payload, auth = auth)


In [26]:
user_json = user_get.json()

In [28]:
for tweet in user_json:
    print(tweet)

{'created_at': 'Thu Aug 15 13:00:01 +0000 2019', 'id': 1161985898155692032, 'id_str': '1161985898155692032', 'full_text': "Before I was Vice President, before I was a Senator — I started my career as an elected official on the New Castle County Council. Local races matter. I'm proud to sign the @runforsomething pledge to support candidates up and down the ballot to invest in our party’s future.", 'truncated': False, 'display_text_range': [0, 274], 'entities': {'hashtags': [], 'symbols': [], 'user_mentions': [{'screen_name': 'runforsomething', 'name': 'Run for Something', 'id': 809420163711909888, 'id_str': '809420163711909888', 'indices': [173, 189]}], 'urls': []}, 'source': '<a href="https://about.twitter.com/products/tweetdeck" rel="nofollow">TweetDeck</a>', 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None, 'in_reply_to_user_id_str': None, 'in_reply_to_screen_name': None, 'user': {'id': 939091, 'id_str': '939091', 'name': 'Joe Biden', 'scr

In [ ]:
today_datetime = dt.datetime.utcnow()
today_date = today_datetime.date()
two_days_prior = dt.date.today() - dt.timedelta(days = 2)

In [44]:
# Functions for converting string date data received from Twitter API into datetime objects
def convert_time(date_string):
    datetime_object = dt.datetime.strptime(date_string, "%a %b %d %H:%M:%S %z %Y")
    time_object = datetime_object.time()
    print(time_object)
    return time_object
def convert_date(date_string):
    datetime_object = dt.datetime.strptime(date_string, "%a %b %d %H:%M:%S %z %Y")
    date_object = datetime_object.date()
    print(date_object)
    return date_object
def convert_datetime(date_string):
    datetime_object = dt.datetime.strptime(date_string, "%a %b %d %H:%M:%S %z %Y")
    print(datetime_object)
    return datetime_object

In [34]:
def time_filter(list_element):
    date_string = list_element["created_at"]
    datetime_object = dt.datetime.strptime(date_string, "%a %b %d %H:%M:%S %z %Y")
    date_object = datetime_object.date()
    
    today_datetime = dt.datetime.utcnow()
    today_date = today_datetime.date()
    two_days_prior = dt.date.today() - dt.timedelta(days = 2)
    
    return date_object <= two_days_prior

In [35]:
user_filtered = list(filter(lambda x: time_filter(x), user_json))

In [37]:
len(user_filtered)

94

In [38]:
import random

In [199]:
tweet_selection = random.choice(user_filtered)

In [200]:
tweet_selection

{'created_at': 'Wed Aug 07 22:21:57 +0000 2019',
 'id': 1159228212465025025,
 'id_str': '1159228212465025025',
 'full_text': 'We are in a battle for the soul of this nation. It’s why I’m running for president. https://t.co/XgVOF7Xt8q',
 'truncated': False,
 'display_text_range': [0, 83],
 'entities': {'hashtags': [],
  'symbols': [],
  'user_mentions': [],
  'urls': [],
  'media': [{'id': 1159224203654180865,
    'id_str': '1159224203654180865',
    'indices': [84, 107],
    'media_url': 'http://pbs.twimg.com/media/EBZmm3EXYAIA267.jpg',
    'media_url_https': 'https://pbs.twimg.com/media/EBZmm3EXYAIA267.jpg',
    'url': 'https://t.co/XgVOF7Xt8q',
    'display_url': 'pic.twitter.com/XgVOF7Xt8q',
    'expanded_url': 'https://twitter.com/JoeBiden/status/1159228212465025025/video/1',
    'type': 'photo',
    'sizes': {'thumb': {'w': 150, 'h': 150, 'resize': 'crop'},
     'small': {'w': 680, 'h': 680, 'resize': 'fit'},
     'medium': {'w': 720, 'h': 720, 'resize': 'fit'},
     'large': {'w'

In [201]:
tweet_dict = {
    'full_text': tweet_selection["full_text"],
    'retweet_count': tweet_selection["retweet_count"],
    'favorite_count': tweet_selection['favorite_count'],
    'created_at': tweet_selection['created_at']
}

In [139]:
def convert_day(date_string):
    datetime_object = dt.datetime.strptime(date_string, "%a %b %d %H:%M:%S %z %Y")
    day = dt.datetime.strftime(datetime_object, "%A")
    return day
def convert_hour(date_string):
    datetime_object = dt.datetime.strptime(date_string, "%a %b %d %H:%M:%S %z %Y")
    hour = dt.datetime.strftime(datetime_object, "%H")
    return hour
def convert_month(date_string):
    datetime_object = dt.datetime.strptime(date_string, "%a %b %d %H:%M:%S %z %Y")
    month = dt.datetime.strftime(datetime_object, "%B")
    return month

In [202]:
tweet_dict['day'] = convert_day(tweet_dict['created_at'])
tweet_dict['hour'] = convert_hour(tweet_dict['created_at'])
tweet_dict['month']  = convert_month(tweet_dict['created_at'])

In [203]:
tweet_dict

{'full_text': 'We are in a battle for the soul of this nation. It’s why I’m running for president. https://t.co/XgVOF7Xt8q',
 'retweet_count': 1694,
 'favorite_count': 7850,
 'created_at': 'Wed Aug 07 22:21:57 +0000 2019',
 'day': 'Wednesday',
 'hour': '22',
 'month': 'August'}

In [205]:
tweet_list = [tweet_dict]

In [207]:
tweet_df = pd.DataFrame(tweet_list)
# tweet_df = pd.get_dummies(tweet_df, columns = ['month', 'day', 'hour'])
tweet_df.head()

,created_at,day,favorite_count,full_text,hour,month,retweet_count
0,Wed Aug 07 22:21:57 +0000 2019,Wednesday,7850,We are in a battle for the soul of this nation. It’s why I’m running for president. https://t.co...,22,August,1694


In [63]:
import nltk
import re
import string
pd.set_option('display.max_colwidth', 100) # To extend column width

stopwords = nltk.corpus.stopwords.words('english')
wn = nltk.WordNetLemmatizer()

In [64]:
def clean_text(text):
    text = "".join([word.lower() for word in text if word not in string.punctuation])
    tokens = re.split('\W+', text)
    text = [wn.lemmatize(word) for word in tokens if word not in stopwords]
    return text

In [209]:
from sklearn.feature_extraction.text import CountVectorizer

# NGramVectorizer
ngram_vect = CountVectorizer(ngram_range=(2,2), analyzer=clean_text)
X_count = ngram_vect.fit_transform(tweet_df['full_text'])

In [210]:
ngram_vect.get_feature_names()

['battle', 'httpstcoxgvof7xt8q', 'nation', 'president', 'running', 'soul']

In [211]:
X_count.toarray()

array([[1, 1, 1, 1, 1, 1]], dtype=int64)

In [92]:
# tweet_df.drop(columns = ["full_text", "created_at"], inplace = True)

In [114]:
import pickle
with open('columns.pkl', 'rb') as f:
   columns_list = pickle.load(f)

In [156]:
# columns_list

In [130]:
null_list = []
for i in range(0, len(columns_list)):
    null_list.append(0)

In [212]:
X_features = dict(zip(columns_list, null_list))

In [213]:
X_features['retweet_count'] = tweet_dict['retweet_count']
X_features['favorite_count'] = tweet_dict['favorite_count']

select_month = tweet_dict['month']
select_day = tweet_dict['day']
select_hour = tweet_dict['hour']
X_features[f'month_{select_month}'] = 1
X_features[f'day_{select_day}'] = 1
X_features[f'hour_{select_hour}'] = 1

In [214]:
for word in ngram_vect.get_feature_names():
    if word in X_features.keys():
        X_features[word] += 1

In [215]:
len(list(X_features.values()))

58604

In [216]:
X_features = np.array(list(X_features.values())).reshape((1, 58604))

In [217]:
X_features.shape

(1, 58604)

In [97]:
from scipy.sparse import csr_matrix

In [218]:
X_sparse = csr_matrix(X_features)

In [219]:
X_sparse

<1x58604 sparse matrix of type '<class 'numpy.int32'>'
	with 11 stored elements in Compressed Sparse Row format>

In [220]:
X_sparse.toarray()

array([[1694, 7850,    0, ...,    0,    0,    0]], dtype=int32)

In [82]:
from sklearn.externals import joblib
scaler_filename = "mas_scaler.save"
scaler = joblib.load(scaler_filename) 

In [222]:
X_scaled = scaler.transform(X_sparse)

In [81]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
encoder.classes_ = np.load('classes.npy')

In [223]:
prediction = model.predict_classes(X_scaled)
prediction_label = encoder.inverse_transform(prediction)
prediction_label

array(['Joe Biden'], dtype=object)

In [247]:
prediction

array([10], dtype=int64)

In [250]:
model.predict_classes(X_scaled)

array([[6.5601955e-09, 1.5463845e-09, 2.1016502e-08, 1.4474209e-07,
        2.7478936e-05, 1.2706663e-06, 6.7904815e-10, 8.6295276e-10,
        4.1059836e-10, 4.1977799e-09, 9.9990618e-01, 7.2136586e-08,
        5.3992815e-05, 1.3098868e-08, 1.8535442e-07, 8.1982780e-06,
        1.2760118e-08, 3.8322306e-10, 6.0801460e-07, 3.2365483e-10,
        2.3451496e-07, 1.6705565e-07, 7.5104492e-09, 1.1033109e-08,
        1.0719514e-07, 1.5677435e-07, 1.6654359e-07, 9.7662951e-07]],
      dtype=float32)

In [248]:
# model.predict_proba(X_scaled)
prediction_prob = model.predict_proba(X_scaled)
prediction_prob_sort = np.sort(prediction_prob)
first_prob = prediction_prob_sort.item(-1)
first_prob
# prediction_labels = encoder.inverse_transform(prediction_prob)
# prediction_labels

0.9999061822891235

In [249]:
prediction_prob

array([[6.5601955e-09, 1.5463845e-09, 2.1016502e-08, 1.4474209e-07,
        2.7478936e-05, 1.2706663e-06, 6.7904815e-10, 8.6295276e-10,
        4.1059836e-10, 4.1977799e-09, 9.9990618e-01, 7.2136586e-08,
        5.3992815e-05, 1.3098868e-08, 1.8535442e-07, 8.1982780e-06,
        1.2760118e-08, 3.8322306e-10, 6.0801460e-07, 3.2365483e-10,
        2.3451496e-07, 1.6705565e-07, 7.5104492e-09, 1.1033109e-08,
        1.0719514e-07, 1.5677435e-07, 1.6654359e-07, 9.7662951e-07]],
      dtype=float32)

In [224]:
help(model)

Help on Sequential in module keras.engine.sequential object:

class Sequential(keras.engine.training.Model)
 |  Sequential(layers=None, name=None)
 |  
 |  Linear stack of layers.
 |  
 |  # Arguments
 |      layers: list of layers to add to the model.
 |  
 |  # Example
 |  
 |  ```python
 |  # Optionally, the first layer can receive an `input_shape` argument:
 |  model = Sequential()
 |  model.add(Dense(32, input_shape=(500,)))
 |  
 |  # Afterwards, we do automatic shape inference:
 |  model.add(Dense(32))
 |  
 |  # This is identical to the following:
 |  model = Sequential()
 |  model.add(Dense(32, input_dim=500))
 |  
 |  # And to the following:
 |  model = Sequential()
 |  model.add(Dense(32, batch_input_shape=(None, 500)))
 |  
 |  # Note that you can also omit the `input_shape` argument:
 |  # In that case the model gets built the first time you call `fit` (or other
 |  # training and evaluation methods).
 |  model = Sequential()
 |  model.add(Dense(32))
 |  model.add(Dense(32